In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy
import sep

In [ ]:
from astropy.io import fits
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

imports and guidelines to make the plots show correctly

In [ ]:
hdulist = fits.open('image.fits')
data = hdulist[0].data

importing the image with astropy and defining the data

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('image raw')

returning the image

In [ ]:
bkg = sep.Background(data)

measuring out a background from the image

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

return an overall noise and mean of the background

In [ ]:
bkg_image = bkg.back()

turn the background into an s-d array 

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('background')

return the background with all the specifications

In [ ]:
bkg_rms = bkg.rms()

turn the noise of the background as a 2-d array

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('background2')

show the 2-d array of the background noise

In [ ]:
data_sub = data - bkg

remove the background

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

set a threshold for the background noise

In [ ]:
len(objects)

return the number of objects found from the threshold

In [ ]:
from matplotlib.patches import Ellipse
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('highlighted_final')

plot the image of the removed background
circle each detected object in red

In [ ]:
objects.dtype.names

list what other parameters could have been found

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

setting a radius for the objects

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

return the first 10 results